# Исследование уровня потребительской лояльности (NPS) клиентов телекоммуникационной компании

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
#Формирование SQL-запроса для выгрузки данных
query = """
SELECT u.user_id,
       u.lt_day, 
       CASE WHEN u.lt_day <= 365 THEN 'новый'
            WHEN u.lt_day > 365 THEN 'старый' END AS is_new,
       u.age, 
       CASE WHEN u.gender_segment = 1 THEN 'женщина'
            WHEN u.gender_segment = 0 THEN 'мужчина' ELSE 'неопределенный' END AS gender_segment,
       u.os_name, 
       u.cpe_type_name, 
       l.country, 
       CASE WHEN l.city = 'НабережныеЧелны' THEN 'Набережные Челны'
            WHEN l.city = 'СанктПетербург' THEN 'Санкт-Петербург'
            WHEN l.city = 'СанктПетербург' THEN 'Санкт-Петербург'
            WHEN l.city = 'НижнийНовгород' THEN 'Нижний Новгород'
            WHEN l.city = 'РостовнаДону' THEN 'Ростов-на-Дону'
            WHEN l.city = 'НижнийТагил' THEN 'Нижний Тагил'
            WHEN l.city = 'УланУдэ' THEN 'Улан-Удэ'    ELSE l.city END AS city,
       SUBSTRING(ast.title, 4) AS age_segment,
       SUBSTRING(ts.title, 4) AS traffic_segment,
       SUBSTRING(ls.title, 4) AS lifetime_segment,
       u.nps_score, 
       CASE WHEN u.nps_score BETWEEN 9 AND 10 THEN 'сторонники'
            WHEN u.nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
            WHEN u.nps_score BETWEEN 0 AND 6 THEN 'критики'
            ELSE NULL END AS nps_group
            
FROM user             AS u
JOIN location         AS l  ON l.location_id = u.location_id
JOIN age_segment      AS ast ON ast.age_gr_id = u.age_gr_id
JOIN traffic_segment  AS ts ON ts.tr_gr_id = u.tr_gr_id
JOIN lifetime_segment AS ls ON ls.lt_gr_id = u.lt_gr_id
"""

In [4]:
try:
       df = pd.read_sql(query, engine)
except Exception as e:
       print(f"Ошибка при выполнении SQL-запроса: {e}")

df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,старый,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,старый,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,старый,44.0,женщина,IOS,SMARTPHONE,Россия,Ростов-на-Дону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,старый,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники


In [5]:
#Проверка, что значения в nps_score находятся в диапазоне от 1 до 10
df['nps_score'].unique()

array([10,  6,  9,  2,  5,  8,  1,  4,  7,  3])

In [6]:
#Проверка, что в поле city отсутствуют неявные дубликаты
df['city'].unique()

array(['Уфа', 'Киров', 'Москва', 'Ростов-на-Дону', 'Рязань', 'Омск',
       'Санкт-Петербург', 'Волгоград', 'Тольятти', 'Казань', 'Самара',
       'Красноярск', 'Екатеринбург', 'Калуга', 'Краснодар', 'Иркутск',
       'Пермь', 'Владимир', 'Ижевск', 'Тюмень', 'Оренбург',
       'Нижний Новгород', 'Брянск', 'Челябинск', 'Астрахань', 'Сургут',
       'Тверь', 'Новосибирск', 'Набережные Челны', 'Махачкала', 'Воронеж',
       'Курск', 'Владивосток', 'Балашиха', 'Пенза', 'Калининград', 'Тула',
       'Саратов', 'Кемерово', 'Белгород', 'Барнаул', 'Чебоксары',
       'Архангельск', 'Томск', 'Ярославль', 'Ульяновск', 'Хабаровск',
       'Грозный', 'Ставрополь', 'Липецк', 'Новокузнецк', 'Якутск',
       'Улан-Удэ', 'Сочи', 'Иваново', 'Нижний Тагил', 'Смоленск',
       'Волжский', 'Магнитогорск', 'Чита', 'Череповец', 'Саранск'],
      dtype=object)

In [7]:
#Проверка, что в поле age_segment нет отрицательных значений
df['age_segment'].unique()

array(['45-54', '55-64', '35-44', '16-24', '25-34', '66 +', 'до 16',
       'n/a'], dtype=object)

In [8]:
#Проверка, что в поле age нет отрицательных значений
df['age'].unique()

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

In [9]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Ссылка на дашборд в Tableau: 

https://public.tableau.com/app/profile/marina.zabolotskikh/viz/NPS_17248507154540/NPS_7?publish=yes